# OpenAI Agents SDK - 설정 관리와 최적화

이 튜토리얼은 OpenAI Agents SDK에서 환경별 설정 관리와 성능/비용 최적화 방법을 다룬다. 개발/스테이징/프로덕션 환경에 맞는 에이전트 설정, 토큰 최적화, 응답 캐싱, 메모리 최적화 등의 핵심 패턴을 학습한다.

In [ ]:
# 필요한 패키지 설치
!pip install openai-agents python-dotenv

---

## 1. 설정 관리 (Configuration Management)

이 챕터에서는 환경별 에이전트 설정을 관리하는 방법을 다룬다. 개발, 스테이징, 프로덕션 환경에 따라 다른 모델, 온도, 토큰 제한 등을 적용하는 패턴을 학습한다.

### 학습 내용

- 다양한 배포 환경을 위한 AgentConfig 데이터클래스 구축
- 환경별 에이전트 생성을 위한 팩토리 패턴
- 환경별 모델 설정, 타임아웃, 비용 제한 관리
- 프로덕션 환경을 위한 설정 관리 패턴

### 환경 설정

In [2]:
import os
from dataclasses import dataclass
from typing import Optional, Dict, Any
from dotenv import load_dotenv
from agents import Agent, ModelSettings, Runner

# 환경 변수 로드
load_dotenv()

MODEL = "gpt-4o-mini"

### AgentConfig 데이터클래스

에이전트 설정을 구조화하기 위한 데이터클래스를 정의한다. 이 클래스는 모델, 온도, 토큰 제한, 타임아웃, 트레이싱, 비용 제한 등의 설정을 포함한다.

In [3]:
@dataclass
class AgentConfig:
    """
    에이전트 배포 환경을 위한 설정 클래스이다.
    """
    model: str                      # 사용할 모델
    temperature: float              # 응답 다양성
    max_tokens: int                 # 최대 출력 토큰
    timeout: float                  # 요청 타임아웃 (초)
    enable_tracing: bool            # 트레이싱 활성화 여부
    cost_limit_per_request: float   # 요청당 비용 제한 (USD)
    
    def __repr__(self):
        return (
            f"AgentConfig(model={self.model}, temp={self.temperature}, "
            f"max_tokens={self.max_tokens}, timeout={self.timeout}s)"
        )

print("AgentConfig 데이터클래스가 정의되었다.")

AgentConfig 데이터클래스가 정의되었다.


### 환경별 설정 팩토리

개발(development), 스테이징(staging), 프로덕션(production) 환경에 맞는 설정을 제공하는 팩토리 클래스를 정의한다.

In [18]:
class ConfiguredAgentFactory:
    """
    환경별 에이전트 생성을 위한 팩토리 클래스이다.
    """
    
    @staticmethod
    def get_config(environment: str = "production") -> AgentConfig:
        """
        특정 환경에 대한 설정을 반환한다.
        
        Args:
            environment: 환경 이름 (development, staging, production)
        
        Returns:
            해당 환경의 AgentConfig
        """
        
        configs = {
            "development": AgentConfig(
                model="gpt-4o-mini",      # 개발용으로 빠르고 저렴한 모델
                temperature=0.7,           # 더 다양한 응답으로 테스트
                max_tokens=1000,
                timeout=30.0,
                enable_tracing=True,       # 개발 시 상세 트레이싱
                cost_limit_per_request=0.01
            ),
            "staging": AgentConfig(
                model="gpt-4o",
                temperature=0.5,
                max_tokens=1500,
                timeout=45.0,
                enable_tracing=True,
                cost_limit_per_request=0.05
            ),
            "production": AgentConfig(
                model="gpt-4o-mini",
                temperature=0.3,           # 프로덕션에서는 더 일관된 응답
                max_tokens=2000,
                timeout=60.0,
                enable_tracing=False,      # 프로덕션에서는 오버헤드 감소
                cost_limit_per_request=0.10
            )
        }
        
        return configs.get(environment, configs["production"])
    
    @staticmethod
    def create_agent(
        name: str, 
        instructions: str, 
        environment: str = None,
        tools: list = None
    ) -> Agent:
        """
        지정된 환경에 맞게 설정된 에이전트를 생성한다.
        
        Args:
            name: 에이전트 이름
            instructions: 에이전트 지시문
            environment: 환경 (None이면 ENVIRONMENT 환경변수 사용)
            tools: 에이전트 도구 목록
        
        Returns:
            설정된 Agent 인스턴스
        """
        
        env = environment or os.getenv("ENVIRONMENT", "production")
        config = ConfiguredAgentFactory.get_config(env)
        
        return Agent(
            name=name,
            instructions=instructions,
            model=config.model,
            model_settings=ModelSettings(
                temperature=config.temperature,
                max_tokens=config.max_tokens
            ),
            tools=tools or []
        )

print("ConfiguredAgentFactory 클래스가 정의되었다.")

ConfiguredAgentFactory 클래스가 정의되었다.


### 환경별 설정 비교

In [19]:
print("=== 환경별 설정 비교 ===")
print()

environments = ["development", "staging", "production"]

print(f"{'환경':<15} {'모델':<15} {'온도':<8} {'토큰':<8} {'타임아웃':<10} {'트레이싱':<10}")
print("-" * 70)

for env in environments:
    config = ConfiguredAgentFactory.get_config(env)
    print(
        f"{env:<15} {config.model:<15} {config.temperature:<8} "
        f"{config.max_tokens:<8} {config.timeout:<10} {str(config.enable_tracing):<10}"
    )

=== 환경별 설정 비교 ===

환경              모델              온도       토큰       타임아웃       트레이싱      
----------------------------------------------------------------------
development     gpt-4o-mini     0.7      1000     30.0       True      
staging         gpt-4o          0.5      1500     45.0       True      
production      gpt-4o-mini     0.3      2000     60.0       False     


### 팩토리를 사용한 에이전트 생성

In [20]:
# 다양한 환경에서 에이전트 생성
print("=== 환경별 에이전트 생성 ===")

for env in ["development", "staging", "production"]:
    agent = ConfiguredAgentFactory.create_agent(
        name=f"{env.capitalize()}Agent",
        instructions="당신은 전문적인 고객 서비스 에이전트입니다.",
        environment=env
    )
    config = ConfiguredAgentFactory.get_config(env)
    print(f"\n[{env}]")
    print(f"  에이전트: {agent.name}")
    print(f"  모델: {config.model}")
    print(f"  설정: {config}")

=== 환경별 에이전트 생성 ===

[development]
  에이전트: DevelopmentAgent
  모델: gpt-4o-mini
  설정: AgentConfig(model=gpt-4o-mini, temp=0.7, max_tokens=1000, timeout=30.0s)

[staging]
  에이전트: StagingAgent
  모델: gpt-4o
  설정: AgentConfig(model=gpt-4o, temp=0.5, max_tokens=1500, timeout=45.0s)

[production]
  에이전트: ProductionAgent
  모델: gpt-4o-mini
  설정: AgentConfig(model=gpt-4o-mini, temp=0.3, max_tokens=2000, timeout=60.0s)


In [21]:
# 프로덕션 에이전트 테스트
print("=== 프로덕션 에이전트 테스트 ===")

prod_agent = ConfiguredAgentFactory.create_agent(
    name="ProductionAgent",
    instructions="당신은 전문적인 고객 서비스 에이전트입니다. 간결하고 정확하게 답변하세요.",
    environment="production"
)

result = await Runner.run(prod_agent, "계정 정보는 어떻게 업데이트하나요?")
print(f"\n[질문] 계정 정보는 어떻게 업데이트하나요?")
print(f"[응답] {result.final_output}")

=== 프로덕션 에이전트 테스트 ===

[질문] 계정 정보는 어떻게 업데이트하나요?
[응답] 계정 정보를 업데이트하려면 다음 단계를 따라주세요:

1. 웹사이트에 로그인합니다.
2. '내 계정' 또는 '프로필' 섹션으로 이동합니다.
3. 수정할 정보를 선택하고 변경합니다.
4. 변경 사항을 저장합니다.

필요한 경우 고객 지원에 문의하시면 추가 도움을 드리겠습니다.


---

## 2. 토큰 최적화 (Token Optimization)

이 챕터에서는 비용과 성능을 최적화하는 기법을 다룬다. 사용 사례별 에이전트 설정, 응답 캐싱, 배치 처리 등의 패턴을 학습한다.

### 학습 내용

- 사용 사례별 최적화된 모델 설정으로 에이전트 생성
- API 호출과 비용을 줄이기 위한 응답 캐싱 구현
- 효율성 향상을 위한 배치 처리
- 성능 모니터링 및 캐시 적중률 분석

In [22]:
import os
import asyncio
import time
from dotenv import load_dotenv
from agents import Agent, Runner, ModelSettings
from typing import Dict, Any, List

# 환경 변수 로드
load_dotenv()

# API 키 확인
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY를 찾을 수 없다. .env 파일을 확인하라.")

if not api_key.startswith("sk-"):
    raise ValueError("올바르지 않은 OpenAI API 키 형식이다.")

print("환경 설정 완료. 토큰 최적화 데모를 시작한다.")

환경 설정 완료. 토큰 최적화 데모를 시작한다.


### OptimizedAgentManager 클래스

사용 사례별 최적화와 응답 캐싱을 제공하는 에이전트 관리자 클래스를 정의한다.

In [23]:
class OptimizedAgentManager:
    """
    최적화된 에이전트 성능을 위한 관리자 클래스이다.
    """
    
    def __init__(self):
        self.response_cache = {}  # 응답 캐시
        self.cache_hits = 0       # 캐시 적중 횟수
        self.cache_misses = 0     # 캐시 미스 횟수
    
    def create_optimized_agent(self, use_case: str) -> Agent:
        """
        특정 사용 사례에 최적화된 에이전트를 생성한다.
        
        Args:
            use_case: 사용 사례 (quick_responses, detailed_analysis, creative_tasks)
        
        Returns:
            최적화된 Agent 인스턴스
        """
        
        optimizations = {
            "quick_responses": {
                "model": "gpt-4o-mini",
                "settings": ModelSettings(
                    temperature=0.1,    # 일관된 짧은 응답
                    max_tokens=150,     # 토큰 제한
                    top_p=0.5
                ),
                "instructions": "간결하고 직접적인 응답을 제공하세요. 짧고 정확하게 답변하세요."
            },
            "detailed_analysis": {
                "model": "gpt-4o",
                "settings": ModelSettings(
                    temperature=0.3,    # 분석적이고 일관된 응답
                    max_tokens=2000,
                    top_p=0.8
                ),
                "instructions": "언어를 효율적으로 사용하면서 포괄적인 분석을 제공하세요."
            },
            "creative_tasks": {
                "model": "gpt-4o",
                "settings": ModelSettings(
                    temperature=0.7,    # 창의적 다양성
                    max_tokens=1000,
                    top_p=0.9
                ),
                "instructions": "창의적이지만 집중된 응답을 제공하세요. 불필요한 부연을 피하세요."
            }
        }
        
        config = optimizations.get(use_case, optimizations["quick_responses"])
        
        return Agent(
            name=f"Optimized_{use_case}",
            model=config["model"],
            model_settings=config["settings"],
            instructions=config["instructions"]
        )
    
    async def execute_with_caching(
        self, 
        agent: Agent, 
        user_input: str
    ) -> Dict[str, Any]:
        """
        반복 쿼리를 위한 응답 캐싱과 함께 실행한다.
        
        Args:
            agent: 실행할 에이전트
            user_input: 사용자 입력
        
        Returns:
            응답과 메타데이터를 담은 딕셔너리
        """
        
        # 간단한 캐시 키 (프로덕션에서는 더 정교한 해싱 사용)
        cache_key = f"{agent.name}:{hash(user_input)}"
        
        if cache_key in self.response_cache:
            self.cache_hits += 1
            return {
                "response": self.response_cache[cache_key],
                "cached": True,
                "execution_time": 0.001  # 캐시 응답 시간
            }
        
        # 실행 및 결과 캐시
        self.cache_misses += 1
        start_time = time.time()
        result = await Runner.run(agent, user_input)
        execution_time = time.time() - start_time
        
        # 응답 캐시
        self.response_cache[cache_key] = result.final_output
        
        return {
            "response": result.final_output,
            "cached": False,
            "execution_time": execution_time,
            "token_usage": result.raw_responses[0].usage.total_tokens if result.raw_responses else 0
        }
    
    async def batch_execute(
        self, 
        agent: Agent, 
        queries: List[str]
    ) -> Dict[str, Any]:
        """
        배치로 여러 쿼리를 효율적으로 실행한다.
        
        Args:
            agent: 실행할 에이전트
            queries: 쿼리 목록
        
        Returns:
            결과와 성능 메트릭을 담은 딕셔너리
        """
        
        start_time = time.time()
        
        # 모든 쿼리를 동시에 실행
        tasks = [self.execute_with_caching(agent, query) for query in queries]
        results = await asyncio.gather(*tasks)
        
        total_time = time.time() - start_time
        
        # 성능 메트릭 계산
        cached_count = sum(1 for r in results if r["cached"])
        total_tokens = sum(r.get("token_usage", 0) for r in results)
        
        return {
            "results": results,
            "performance": {
                "total_time": total_time,
                "queries_processed": len(queries),
                "cache_hit_rate": cached_count / len(queries) * 100,
                "total_tokens": total_tokens,
                "avg_time_per_query": total_time / len(queries)
            }
        }
    
    def get_cache_stats(self) -> Dict[str, Any]:
        """캐시 통계를 반환한다."""
        total = self.cache_hits + self.cache_misses
        return {
            "cache_hits": self.cache_hits,
            "cache_misses": self.cache_misses,
            "total_requests": total,
            "hit_rate": (self.cache_hits / total * 100) if total > 0 else 0,
            "cache_size": len(self.response_cache)
        }

print("OptimizedAgentManager 클래스가 정의되었다.")

OptimizedAgentManager 클래스가 정의되었다.


### 사용 사례별 에이전트 비교

In [24]:
# 사용 사례별 설정 비교
print("=== 사용 사례별 에이전트 설정 ===")
print()

manager = OptimizedAgentManager()

use_cases = ["quick_responses", "detailed_analysis", "creative_tasks"]

print(f"{'사용 사례':<20} {'모델':<15} {'온도':<8} {'최대 토큰':<12}")
print("-" * 60)

for use_case in use_cases:
    agent = manager.create_optimized_agent(use_case)
    # ModelSettings에서 설정 추출
    settings = agent.model_settings
    print(
        f"{use_case:<20} {agent.model:<15} "
        f"{settings.temperature:<8} {settings.max_tokens:<12}"
    )

=== 사용 사례별 에이전트 설정 ===

사용 사례                모델              온도       최대 토큰       
------------------------------------------------------------
quick_responses      gpt-4o-mini     0.1      150         
detailed_analysis    gpt-4o          0.3      2000        
creative_tasks       gpt-4o          0.7      1000        


### 캐싱과 배치 처리 데모

In [25]:
async def demonstrate_performance_optimization():
    """
    성능 최적화 기법을 시연하는 함수이다.
    """
    
    print("성능 최적화 시연")
    print("=" * 50)
    
    manager = OptimizedAgentManager()
    
    # 빠른 응답용 에이전트 생성
    quick_agent = manager.create_optimized_agent("quick_responses")
    
    # 테스트 쿼리 (중복 포함)
    quick_queries = [
        "2+2는?",
        "프랑스의 수도는?",
        "AI 정의",
        "2+2는?",           # 중복 - 캐시 테스트
        "프랑스의 수도는?"   # 중복 - 캐시 테스트
    ]
    
    # 캐싱과 함께 빠른 응답 테스트
    print("\n--- 빠른 응답 최적화 (캐싱 포함) ---")
    quick_results = await manager.batch_execute(quick_agent, quick_queries)
    
    print(f"처리된 쿼리: {quick_results['performance']['queries_processed']}개")
    print(f"총 시간: {quick_results['performance']['total_time']:.3f}초")
    print(f"캐시 적중률: {quick_results['performance']['cache_hit_rate']:.1f}%")
    print(f"쿼리당 평균 시간: {quick_results['performance']['avg_time_per_query']:.3f}초")
    print(f"총 토큰: {quick_results['performance']['total_tokens']}")
    
    # 개별 결과 표시
    print("\n[개별 결과]")
    for i, (query, result) in enumerate(zip(quick_queries, quick_results['results'])):
        status = "[캐시]" if result['cached'] else "[실행]"
        print(f"{i+1}. {status} {query} -> {result['response'][:50]}...")
    
    return manager

# 데모 실행
manager = await demonstrate_performance_optimization()

성능 최적화 시연

--- 빠른 응답 최적화 (캐싱 포함) ---
처리된 쿼리: 5개
총 시간: 1.769초
캐시 적중률: 0.0%
쿼리당 평균 시간: 0.354초
총 토큰: 234

[개별 결과]
1. [실행] 2+2는? -> 4입니다....
2. [실행] 프랑스의 수도는? -> 파리입니다....
3. [실행] AI 정의 -> AI(인공지능)는 인간의 지능을 모방하여 학습, 문제 해결, 의사결정 등을 수행하는 컴퓨터...
4. [실행] 2+2는? -> 4입니다....
5. [실행] 프랑스의 수도는? -> 파리입니다....


In [26]:
# 캐시 통계 확인
print("\n=== 캐시 통계 ===")
stats = manager.get_cache_stats()
for key, value in stats.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.1f}%")
    else:
        print(f"  {key}: {value}")


=== 캐시 통계 ===
  cache_hits: 0
  cache_misses: 5
  total_requests: 5
  hit_rate: 0.0%
  cache_size: 3


---

## 3. 메모리 최적화 (Memory Optimization)

이 챕터에서는 대화 메모리 최적화 기법을 다룬다. 긴 대화에서 토큰을 효율적으로 관리하고 중요한 컨텍스트를 보존하는 방법을 학습한다.

### 학습 내용

- 대화 기록 길이 관리를 위한 ContextOptimizer 구축
- 모델 컨텍스트 제한 내에서의 토큰 기반 트리밍
- 중복 정보 제거하면서 중요 컨텍스트 보존
- 긴 세션 대화를 위한 성능 최적화

In [28]:
import os
from dotenv import load_dotenv
from typing import List, Dict, Any

### ContextOptimizer 클래스

대화 컨텍스트를 최적화하는 클래스를 정의한다. 메시지 수와 토큰 수 기반의 트리밍 기능을 제공한다.

In [29]:
class ContextOptimizer:
    """
    대화 컨텍스트를 최적화하여 성능을 향상시키는 클래스이다.
    """
    
    def __init__(self, max_context_length: int = 10):
        """
        ContextOptimizer를 초기화한다.
        
        Args:
            max_context_length: 유지할 최대 메시지 수
        """
        self.max_context_length = max_context_length
    
    def optimize_conversation_history(self, history: List[Dict]) -> List[Dict]:
        """
        성능 유지를 위해 대화 기록을 최적화한다.
        시스템 메시지는 보존하고 최근 메시지를 유지한다.
        
        Args:
            history: 대화 기록 리스트
        
        Returns:
            최적화된 대화 기록
        """
        
        if len(history) <= self.max_context_length:
            return history
        
        # 시스템 메시지와 다른 메시지 분리
        system_messages = [msg for msg in history if msg.get("role") == "system"]
        other_messages = [msg for msg in history if msg.get("role") != "system"]
        
        # 최근 메시지 유지
        keep_count = self.max_context_length - len(system_messages)
        recent_messages = other_messages[-keep_count:]
        
        return system_messages + recent_messages
    
    def estimate_token_count(self, text: str) -> int:
        """
        토큰 수를 대략적으로 추정한다.
        
        Args:
            text: 추정할 텍스트
        
        Returns:
            추정 토큰 수
        """
        # 간단한 근사: 약 4자당 1토큰 (영어 기준)
        # 한글은 약 2자당 1토큰으로 더 정확함
        return len(text) // 3
    
    def trim_by_token_limit(
        self, 
        history: List[Dict], 
        max_tokens: int = 4000
    ) -> List[Dict]:
        """
        추정 토큰 수에 따라 대화 기록을 트리밍한다.
        최근 메시지를 우선적으로 유지한다.
        
        Args:
            history: 대화 기록 리스트
            max_tokens: 최대 토큰 수
        
        Returns:
            트리밍된 대화 기록
        """
        
        total_tokens = 0
        trimmed_history = []
        
        # 역순으로 처리하여 최근 메시지 유지
        for message in reversed(history):
            content = str(message.get("content", ""))
            message_tokens = self.estimate_token_count(content)
            
            if total_tokens + message_tokens <= max_tokens:
                trimmed_history.insert(0, message)
                total_tokens += message_tokens
            else:
                # 시스템 메시지는 항상 포함
                if message.get("role") == "system":
                    trimmed_history.insert(0, message)
                break
        
        return trimmed_history
    
    def get_optimization_stats(
        self, 
        original: List[Dict], 
        optimized: List[Dict]
    ) -> Dict[str, Any]:
        """
        최적화 통계를 반환한다.
        
        Args:
            original: 원본 대화 기록
            optimized: 최적화된 대화 기록
        
        Returns:
            최적화 통계 딕셔너리
        """
        original_tokens = sum(
            self.estimate_token_count(str(m.get("content", ""))) 
            for m in original
        )
        optimized_tokens = sum(
            self.estimate_token_count(str(m.get("content", ""))) 
            for m in optimized
        )
        
        return {
            "original_messages": len(original),
            "optimized_messages": len(optimized),
            "messages_removed": len(original) - len(optimized),
            "original_tokens": original_tokens,
            "optimized_tokens": optimized_tokens,
            "tokens_saved": original_tokens - optimized_tokens,
            "reduction_rate": (1 - optimized_tokens / original_tokens) * 100 if original_tokens > 0 else 0
        }

print("ContextOptimizer 클래스가 정의되었다.")

ContextOptimizer 클래스가 정의되었다.


### 컨텍스트 최적화 데모

In [30]:
def demonstrate_context_optimization():
    """
    컨텍스트 최적화를 시연하는 함수이다.
    """
    
    print("=== 컨텍스트 최적화 데모 ===")
    
    optimizer = ContextOptimizer(max_context_length=5)
    
    # 긴 대화 기록 시뮬레이션
    history = [
        {"role": "system", "content": "당신은 도움이 되는 어시스턴트입니다."},
        {"role": "user", "content": "안녕하세요!"},
        {"role": "assistant", "content": "안녕하세요! 무엇을 도와드릴까요?"},
        {"role": "user", "content": "농담 하나 해주세요."},
        {"role": "assistant", "content": "왜 프로그래머는 자연을 싫어할까요? 버그가 너무 많아서요!"},
        {"role": "user", "content": "하나 더요."},
        {"role": "assistant", "content": "과학자들은 왜 원자를 믿지 않을까요? 모든 것을 만들어내거든요!"},
        {"role": "user", "content": "감사합니다!"},
        {"role": "assistant", "content": "천만에요! 더 도움이 필요하시면 말씀해주세요."},
        {"role": "user", "content": "파이썬에 대해 알려주세요."},
        {"role": "assistant", "content": "파이썬은 읽기 쉽고 다재다능한 프로그래밍 언어입니다. 웹 개발, 데이터 과학, AI 등에 널리 사용됩니다."},
    ]
    
    print(f"\n원본 대화 길이: {len(history)} 메시지")
    
    # 메시지 수 기반 최적화
    optimized_by_count = optimizer.optimize_conversation_history(history)
    print(f"메시지 수 기반 최적화 후: {len(optimized_by_count)} 메시지")
    
    # 토큰 기반 트리밍
    trimmed_by_tokens = optimizer.trim_by_token_limit(history, max_tokens=100)
    print(f"토큰 기반 트리밍 후 (100 토큰): {len(trimmed_by_tokens)} 메시지")
    
    return optimizer, history, optimized_by_count

optimizer, history, optimized = demonstrate_context_optimization()

=== 컨텍스트 최적화 데모 ===

원본 대화 길이: 11 메시지
메시지 수 기반 최적화 후: 5 메시지
토큰 기반 트리밍 후 (100 토큰): 11 메시지


In [31]:
# 최적화 통계
print("\n=== 최적화 통계 ===")
stats = optimizer.get_optimization_stats(history, optimized)
for key, value in stats.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.1f}%")
    else:
        print(f"  {key}: {value}")


=== 최적화 통계 ===
  original_messages: 11
  optimized_messages: 5
  messages_removed: 6
  original_tokens: 73
  optimized_tokens: 39
  tokens_saved: 34
  reduction_rate: 46.6%


In [32]:
# 최적화된 대화 내용 확인
print("\n=== 최적화된 대화 내용 ===")
for msg in optimized:
    role = msg['role']
    content = msg['content'][:50] + "..." if len(msg['content']) > 50 else msg['content']
    print(f"  [{role}] {content}")


=== 최적화된 대화 내용 ===
  [system] 당신은 도움이 되는 어시스턴트입니다.
  [user] 감사합니다!
  [assistant] 천만에요! 더 도움이 필요하시면 말씀해주세요.
  [user] 파이썬에 대해 알려주세요.
  [assistant] 파이썬은 읽기 쉽고 다재다능한 프로그래밍 언어입니다. 웹 개발, 데이터 과학, AI 등에 ...


### 최적화 모범 사례

1. 설정 관리
   - 환경별(개발/스테이징/프로덕션) 설정 분리
   - 데이터클래스로 설정 구조화
   - 팩토리 패턴으로 에이전트 생성 표준화

2. 토큰 최적화
   - 사용 사례별 최적화된 모델 설정
   - 응답 캐싱으로 중복 요청 방지
   - 배치 처리로 처리량 향상

3. 메모리 최적화
   - 대화 기록 길이 제한
   - 토큰 기반 트리밍
   - 시스템 메시지 항상 보존

4. 비용 관리
   - 요청당 비용 제한 설정
   - 토큰 사용량 모니터링
   - 개발 시 저렴한 모델 사용

---

## 마무리

이 튜토리얼에서는 OpenAI Agents SDK의 설정 관리와 최적화 기법을 다루었다. 다음과 같은 내용을 학습하였다:

1. **설정 관리 (Configuration Management)**
   - AgentConfig 데이터클래스로 설정 구조화
   - ConfiguredAgentFactory로 환경별 에이전트 생성
   - 개발/스테이징/프로덕션 환경별 최적 설정

2. **토큰 최적화 (Token Optimization)**
   - OptimizedAgentManager로 사용 사례별 에이전트 관리
   - 응답 캐싱으로 중복 API 호출 방지
   - 배치 처리로 동시 쿼리 효율적 처리

3. **메모리 최적화 (Memory Optimization)**
   - ContextOptimizer로 대화 기록 관리
   - 메시지 수 및 토큰 기반 트리밍
   - 시스템 메시지 보존하며 최근 컨텍스트 유지

이러한 패턴을 적용하면 비용을 절감하고, 성능을 향상시키며, 다양한 환경에서 일관된 에이전트 동작을 보장할 수 있다.